In [12]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# from RoBERTa import *
# model = CustomRobertaModel()


In [14]:
# from transformers import RobertaTokenizer
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# input_tokens =tokenizer("test and test.", return_tensors="pt", truncation=True, padding=True, max_length=256)
# input_tokens = input_tokens.to(device)


In [33]:
from transformers import RobertaTokenizer, RobertaModel

# Load pre-trained model and tokenizer
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
try_model = RobertaModel.from_pretrained(model_name)

# Prepare the input text
text = "This is an example sentence."
encoded_input = tokenizer(text, return_tensors="pt")

# Feed the input into the model
output = try_model(**encoded_input)

# Extract the CLS token embedding
cls_embedding = output.last_hidden_state[:, 0, :]

print(cls_embedding)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-1.0255e-01,  1.0369e-01, -1.0960e-02, -5.4950e-02,  1.2176e-01,
         -3.2302e-03, -2.3434e-02,  4.3359e-02,  5.0298e-02, -8.6572e-02,
         -2.3857e-02,  3.4339e-02,  4.3145e-02, -7.7734e-02,  5.4695e-02,
         -1.0053e-01, -4.0279e-02, -5.7762e-03, -1.4725e-02, -4.9593e-02,
         -1.3021e-01,  2.5552e-02,  2.0522e-02,  1.0200e-01, -6.3678e-02,
          8.8177e-02,  1.4348e-01,  5.3061e-02, -7.1654e-02,  5.3694e-02,
         -6.8900e-02, -9.9970e-02,  8.2682e-02, -1.3511e-02,  2.0117e-02,
          1.3400e-01,  2.7797e-03, -8.3667e-04, -5.8601e-02,  5.1972e-02,
          1.2828e-02,  2.1757e-01,  1.7009e-02, -4.6349e-02,  6.8069e-02,
          3.1592e-02,  3.6040e-02, -2.5366e-02,  1.2174e-02, -1.6848e-02,
          4.5981e-03,  5.4572e-02, -2.1784e-02,  5.7348e-02, -1.3289e-01,
          6.8254e-02,  3.0545e-02,  7.2083e-02,  8.2089e-02, -1.2538e-01,
         -3.0586e-02, -1.8470e-01, -6.9079e-02, -6.0674e-02,  5.5125e-02,
         -1.0963e-01, -3.8253e-02,  5.

In [16]:
# model.to(device)
# model(input_tokens)

In [17]:
# from transformers import RobertaTokenizer, RobertaModelWithHeads
# from transformers.adapters.composition import Stack

# # Load pre-trained model and tokenizer
# model_name = "roberta-base"
# tokenizer = RobertaTokenizer.from_pretrained(model_name)
# model = RobertaModelWithHeads.from_pretrained(model_name)

# # Add a new adapter
# adapter_name = "my_adapter"
# model.add_adapter(adapter_name)

# # Add a classification head for your task
# num_labels = 2  # Change this value according to your task
# model.add_classification_head(adapter_name, num_labels=num_labels)

# # Set the adapters to be used in training
# model.train_adapter([adapter_name])

# # You can now train your model with your dataset and use it for inference.


In [18]:
# model.train()

In [19]:
# text = "This movie was amazing!"
# cls = model(text)
# print(cls)

In [41]:
import torch
import argparse
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from load_wiki_dataset import wikiData
from RoBERTa import CustomRobertaModel
from losses import align_loss, uniform_loss
from transformers import RobertaTokenizer, RobertaModel

# Check for GPU availability and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def parse_arguments():
    parser = argparse.ArgumentParser()
    parser.add_argument('--epochs', type=int, default=100, help='Number of training epochs')
    parser.add_argument('--batch_size', type=int, default=8, help='Batch size')
    parser.add_argument('--learning_rate', type=float, default=1e-4, help='Learning rate')
    parser.add_argument('--data_path', type=str, default='../data/wiki1m_for_simcse.txt', help='Path to the dataset')
    return parser.parse_args()

def train_wiki(args,model):
    # Load dataset
    with open(args.data_path, 'r', encoding='UTF-8') as f:
        input_text = f.readlines()

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')  # ADD THIS LINE

    wiki = wikiData(input_text, tokenizer)  # PASS tokenizer TO wikiData
    train_params = {'batch_size': args.batch_size, 'shuffle': True, 'num_workers': 0}
    trainloader = DataLoader(wiki, **train_params)
    
    # Initialize model
    model.to(device)
    model.train()

    # Set up the optimizer
    optimizer = AdamW(model.parameters(), lr=args.learning_rate)
    # Training loop
    # for epoch in range(args.epochs):
    #     epoch_loss = 0
    #     for batch in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{args.epochs}"):
    #         optimizer.zero_grad()
    #         batch=tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=256)
    #         batch = [torch.tensor(item).to(device) for item in batch]  # Move batch to device (GPU), change it according to dataset
    #         loss, _ = model(batch)
    #         loss.backward()
    #         optimizer.step()
    #         epoch_loss += loss.item()

    #     print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")
    for epoch in range(args.epochs):
            epoch_loss = 0
            for batch in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{args.epochs}"):
                optimizer.zero_grad()
                batch = {k: v.to(device) for k, v in batch.items()}  # move batch to device
                
                loss, _ = model(batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                # print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")
    print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")

    return model

In [24]:
args = argparse.Namespace()
args.epochs = 1
args.batch_size = 8
args.learning_rate = 1e-4
args.data_path = "../data/wiki1m_for_simcse.txt"

In [25]:
trained_model = train_wiki(args)
    # Save the trained model if needed
trained_model.save_pretrained('./trained_model_wiki')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

Epoch 1 Loss: -5.239096641540528e-06
Epoch 1 Loss: -1.0496516704559326e-05


Epoch 1/1:   0%|          | 4/125000 [00:00<6:04:56,  5.71it/s]

Epoch 1 Loss: -1.6157252311706543e-05
Epoch 1 Loss: -2.1675745010375977e-05


Epoch 1/1:   0%|          | 6/125000 [00:01<5:57:09,  5.83it/s]

...

Epoch 1/1:   0%|          | 72/125000 [00:12<5:46:53,  6.00it/s]

Epoch 1 Loss: -0.0009451745309829711
Epoch 1 Loss: -0.0009658331117630005


Epoch 1/1:   0%|          | 73/125000 [00:12<5:55:16,  5.86it/s]

Epoch 1 Loss: -0.0009881509828567506


KeyboardInterrupt: 

In [24]:
from RoBERTa import CustomRobertaModel

In [25]:
wiki_model2 = CustomRobertaModel()
wiki_model2.load_state_dict(torch.load('1wiki.pth'))


c:\Users\Ben\anaconda3\envs\545\lib\site-packages\transformers\adapters\models\roberta\adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
c:\Users\Ben\anaconda3\envs\545\lib\site-packages\transformers\adapters\models\roberta\adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertF

<All keys matched successfully>

In [7]:
from transformers import AutoModel,RobertaTokenizer # Load the trained model 

wiki_model = AutoModel.from_pretrained('./trained_model_wiki')

Some weights of the model checkpoint at ./trained_model_wiki were not used when initializing RobertaModel: ['roberta.roberta.encoder.layer.6.output.adapters.my_adapter.adapter_down.0.weight', 'roberta_m.roberta.encoder.layer.8.attention.self.value.bias', 'roberta.roberta.encoder.layer.4.output.dense.weight', 'roberta_m.roberta.encoder.layer.8.output.LayerNorm.weight', 'roberta_m.roberta.encoder.layer.11.output.adapters.my_adapter.adapter_down.0.weight', 'roberta.roberta.encoder.layer.3.output.adapters.my_adapter.adapter_down.0.weight', 'roberta_m.roberta.encoder.layer.9.attention.self.key.bias', 'roberta_m.roberta.encoder.layer.4.intermediate.dense.weight', 'roberta_m.roberta.encoder.layer.4.attention.self.query.weight', 'roberta.roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.roberta.encoder.layer.9.attention.output.dense.bias', 'roberta_m.roberta.encoder.layer.5.output.dense.weight', 'roberta.roberta.encoder.layer.8.attention.self.value.weight', 'roberta_m.robert

In [1]:
from load_allsides_dataset import allsidesData
from torch.utils.data import DataLoader
import torch
import argparse
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from load_wiki_dataset import wikiData
from losses import align_loss, uniform_loss
from transformers import RobertaTokenizer, RobertaModel
def train_allsides(args, model_to_train):
    # Load dataset
    with open(args.data_path, 'r', encoding='UTF-8') as f:
        input_text = f.readlines()

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')  # ADD THIS LINE

    allsides = allsidesData(input_text, tokenizer)  # PASS tokenizer TO wikiData
    train_params = {'batch_size': args.batch_size, 'shuffle': True, 'num_workers': 0}
    trainloader = DataLoader(allsides, **train_params)
    
    # Initialize model
    model = model_to_train
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.train()

    # Set up the optimizer
    optimizer = AdamW(model.parameters(), lr=args.learning_rate)
    # Training loop
    # for epoch in range(args.epochs):
    #     epoch_loss = 0
    #     for batch in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{args.epochs}"):
    #         optimizer.zero_grad()
    #         batch=tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=256)
    #         batch = [torch.tensor(item).to(device) for item in batch]  # Move batch to device (GPU), change it according to dataset
    #         loss, _ = model(batch)
    #         loss.backward()
    #         optimizer.step()
    #         epoch_loss += loss.item()

    #     print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")
    for epoch in range(args.epochs):
            epoch_loss = 0
            for batch in tqdm(trainloader, desc=f"Epoch {epoch + 1}/{args.epochs}"):
                optimizer.zero_grad()
                batch = {k: v.to(device) for k, v in batch.items()}  # move batch to device
                # print(model(batch))
                loss, cls = model(batch)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                # print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")
    print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(trainloader)}")

    return model

In [32]:
import argparse

args = argparse.Namespace()
args.epochs = 1
args.batch_size = 8
args.learning_rate = 1e-4
args.data_path = "../data/allsides.jsonl"

In [33]:
allsides_wiki_model = train_allsides(args,wiki_model2)
    # Save the trained model if needed
allsides_wiki_model.save_pretrained('./allsides_wiki_model')

Epoch 1/1: 100%|██████████| 65720/65720 [3:01:13<00:00,  6.04it/s]  


Epoch 1 Loss: -3.609909778521399


In [34]:
torch.save(allsides_wiki_model.state_dict(), 'allsides_wiki_model.pth')


In [37]:
args = argparse.Namespace()
args.epochs = 1
args.batch_size = 8
args.learning_rate = 1e-4
args.data_path = "../data/wiki1m_for_simcse.txt"

In [42]:
allsides_wiki2_model = train_wiki(args,allsides_wiki_model)
    # Save the trained model if needed
allsides_wiki2_model.save_pretrained('./allsides_wiki2_model')
torch.save(allsides_wiki2_model.state_dict(), 'allsides_wiki2_model.pth')


Epoch 1/1:  70%|██████▉   | 86938/125000 [3:55:10<1:42:57,  6.16it/s]


KeyboardInterrupt: 

In [3]:
from RoBERTa import CustomRobertaModel
wiki_model = CustomRobertaModel()
wiki_model.load_state_dict(torch.load('1wiki.pth'))
allsides_wiki_model = CustomRobertaModel()
allsides_wiki_model.load_state_dict(torch.load('allsides_wiki_model.pth'))

c:\Users\Ben\anaconda3\envs\545\lib\site-packages\transformers\adapters\models\roberta\adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
c:\Users\Ben\anaconda3\envs\545\lib\site-packages\transformers\adapters\models\roberta\adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertF

<All keys matched successfully>